In [1]:
import pandas as pd
import numpy as np

In [4]:
customers = pd.read_csv('https://raw.githubusercontent.com/Rohanrathod7/my-ds-projects/main/02_data_manipulation/project_order_pipeline/Data/customers.csv')
print(customers.head())
products = pd.read_csv('https://raw.githubusercontent.com/Rohanrathod7/my-ds-projects/main/02_data_manipulation/project_order_pipeline/Data/products.csv')
print(products.head())
orders = pd.read_csv('https://raw.githubusercontent.com/Rohanrathod7/my-ds-projects/main/02_data_manipulation/project_order_pipeline/Data/orders.csv', parse_dates=['OrderDate'])
print(orders.head())
order_items = pd.read_csv('https://raw.githubusercontent.com/Rohanrathod7/my-ds-projects/main/02_data_manipulation/project_order_pipeline/Data/order_items.csv')
print(order_items.head())
shippings = pd.read_csv('https://raw.githubusercontent.com/Rohanrathod7/my-ds-projects/main/02_data_manipulation/project_order_pipeline/Data/shippings.csv', parse_dates=['ShippingDate'])
print(shippings.head())

   CustomerID   CustomerName                        Email  \
0           1   William Tran      hessspencer@herring.net   
1           2     Allen Ward  marieruiz@flores-conner.com   
2           3  Shannon Perez        brownstacey@yahoo.com   
3           4  Vanessa Walls   michaelmoore@roy-green.com   
4           5     Shawn Mack      tiffanypaul@morales.com   

                       Country  
0            Equatorial Guinea  
1                       Latvia  
2                     Suriname  
3  French Southern Territories  
4                    Macedonia  
   ProductID         ProductName     Category  UnitPrice
0          1   Stock Electronics  Electronics      17.38
1          2  President Clothing     Clothing     126.22
2          3         Doctor Home         Home     369.55
3          4          Fill Books        Books      48.03
4          5           Some Toys         Toys      20.73
   OrderID  CustomerID  OrderDate
0        1          18 2025-04-10
1        2          88 20

In [5]:
df = orders.merge(customers, on='CustomerID', how='left')
df = df.merge(order_items, on='OrderID', how='left')
df = df.merge(products, on='ProductID', how='left')
df = df.merge(shippings, on='OrderID', how='left')

print(df.head(10))

   OrderID  CustomerID  OrderDate    CustomerName  \
0        1          18 2025-04-10     Terri White   
1        1          18 2025-04-10     Terri White   
2        1          18 2025-04-10     Terri White   
3        1          18 2025-04-10     Terri White   
4        1          18 2025-04-10     Terri White   
5        2          88 2025-03-10      Mark Banks   
6        2          88 2025-03-10      Mark Banks   
7        2          88 2025-03-10      Mark Banks   
8        2          88 2025-03-10      Mark Banks   
9        3          60 2025-02-13  Christian Cook   

                            Email                            Country  \
0             fbrown@woodward.org           Turks and Caicos Islands   
1             fbrown@woodward.org           Turks and Caicos Islands   
2             fbrown@woodward.org           Turks and Caicos Islands   
3             fbrown@woodward.org           Turks and Caicos Islands   
4             fbrown@woodward.org           Turks and Ca

In [10]:
df['OrderTotal'] = df['Quantity'] * df['UnitPrice_x']
print(df['OrderTotal'].head())
df['ShippingDelay'] = (df['ShippingDate'] - df['OrderDate']).dt.days
print(df['ShippingDelay'].head())

0    1915.28
1     108.71
2     126.22
3     498.68
4     227.42
Name: OrderTotal, dtype: float64
0    3
1    3
2    3
3    3
4    3
Name: ShippingDelay, dtype: int64


In [11]:
late_shipments = df.query('ShippingDelay > 5')
print(late_shipments.head())
top_customers = df.groupby(['CustomerID', 'CustomerName'])['OrderTotal'].sum().nlargest(10)
print(top_customers)
top_products = df.groupby('ProductName')['Quantity'].sum().nlargest(10)
print(top_products)

    OrderID  CustomerID  OrderDate    CustomerName                  Email  \
5         2          88 2025-03-10      Mark Banks   zgilmore@hotmail.com   
6         2          88 2025-03-10      Mark Banks   zgilmore@hotmail.com   
7         2          88 2025-03-10      Mark Banks   zgilmore@hotmail.com   
8         2          88 2025-03-10      Mark Banks   zgilmore@hotmail.com   
17        5          59 2025-02-16  Elizabeth Diaz  amandasmith@rojas.com   

                              Country  ProductID  Quantity  UnitPrice_x  \
5   Heard Island and McDonald Islands         36         5       113.71   
6   Heard Island and McDonald Islands          9         1       212.66   
7   Heard Island and McDonald Islands          5         4        20.73   
8   Heard Island and McDonald Islands         35         4       138.62   
17                          Lithuania         33         3       269.40   

         ProductName Category  UnitPrice_y ShippingDate  ShippingDelay  \
5       Foll

In [13]:
revenue_by_category = df.groupby('Category')['OrderTotal'].sum().sort_values(ascending=False)
print(revenue_by_category)
revenue_pivot = df.pivot_table(index='CustomerName', columns='Category', values='OrderTotal', aggfunc='sum', fill_value=0)
print(revenue_pivot)

Category
Home           160802.46
Electronics    142680.97
Clothing       140182.94
Books          124629.82
Toys            72389.04
Name: OrderTotal, dtype: float64
Category         Books  Clothing  Electronics     Home     Toys
CustomerName                                                   
Allen Ward     1773.20    844.42         0.00   498.68   896.26
Amanda Ho      1087.16    872.37       473.52     0.00   170.94
Amber Daniel   1427.18   1112.79      4879.28  3359.42  2442.47
April Lucas       0.00      0.00         0.00  2326.02     0.00
April Price    3760.02   1311.32      2921.08     0.00  1132.47
...                ...       ...          ...      ...      ...
Thomas Lester  1316.40    543.55      2463.20  1847.75  2136.37
Tina Cook MD   2809.88    915.86      2949.54  4108.91  2438.97
Tyler Wilson    192.12   2743.97      3532.68  3563.22  1846.98
Vanessa Walls  3743.46   1541.82         0.00   523.56   699.79
William Tran    144.09      0.00      2335.51  1496.04     0.00



Reshape & Clean

In [15]:
reshaped = df[['OrderID', 'ProductName', 'Quantity']].melt(id_vars='OrderID', var_name='Detail', value_name='Value')
print(reshaped)

      OrderID       Detail               Value
0           1  ProductName            Big Home
1           1  ProductName     Amount Clothing
2           1  ProductName  President Clothing
3           1  ProductName          Third Home
4           1  ProductName        Follow Books
...       ...          ...                 ...
1801      298     Quantity                   4
1802      298     Quantity                   1
1803      299     Quantity                   5
1804      299     Quantity                   3
1805      300     Quantity                   3

[1806 rows x 3 columns]


In [17]:
df['ShippingDateFilled'] = df['ShippingDate'].fillna(method='ffill')
print(df["ShippingDateFilled"])

0     2025-04-13
1     2025-04-13
2     2025-04-13
3     2025-04-13
4     2025-04-13
         ...    
898   2025-07-19
899   2025-07-19
900   2025-04-05
901   2025-04-05
902   2025-03-08
Name: ShippingDateFilled, Length: 903, dtype: datetime64[ns]


/tmp/ipython-input-3868761891.py:1: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['ShippingDateFilled'] = df['ShippingDate'].fillna(method='ffill')


In [18]:
print("\nTop 10 Customers by Spend:\n", top_customers)
print("\nTop 10 Products by Quantity:\n", top_products)
print("\nRevenue by Category:\n", revenue_by_category)


Top 10 Customers by Spend:
 CustomerID  CustomerName     
98          Brittany Smith       22805.14
79          Jacob Cline          20121.04
65          Sylvia Nelson        16500.17
80          Nathaniel Fox        16289.17
68          Daniel Butler        16124.46
61          Grant Garcia         16048.00
10          Brandon Howard       15343.62
9           Christian Nichols    14955.92
94          Pamela Perez         14328.09
100         Elizabeth West       13996.12
Name: OrderTotal, dtype: float64

Top 10 Products by Quantity:
 ProductName
Big Home                  77
Per Home                  77
Follow Electronics        73
Mission Clothing          73
Congress Books            72
Experience Electronics    69
Amount Clothing           67
Economy Electronics       67
Some Toys                 67
West Toys                 67
Name: Quantity, dtype: int64

Revenue by Category:
 Category
Home           160802.46
Electronics    142680.97
Clothing       140182.94
Books          1246